In [1]:
from sudoku import Sudoku
import numpy as np

In [2]:
from deep_sudoku.utils import sudoku_utils

In [3]:
string = "000000010400000000020000000000050407008000300001090000300400200050100000000806000"
board = sudoku_utils.load_string(string)
solved = sudoku_utils.solve_sudoku(board)

In [4]:
def make_random_move(board, solved, correct = True):
    temp_board = board.copy()
    possible_moves = np.argwhere(temp_board == 0)
    move_index = np.random.choice(range(len(possible_moves)))
    pos_to_play = possible_moves[move_index]

    correct_number = int(solved[pos_to_play[0], pos_to_play[1]])
    
    if correct:
        new_number = correct_number
    else:
        new_number = np.random.choice([x for x in range(1,9) if x!= correct_number])
        
    temp_board[pos_to_play[0], pos_to_play[1]] = new_number
    
    return temp_board

checking = False


for i in range(1000):
    new_board = board.copy()
    
    for j in range(20):
        new_board = make_random_move(new_board, solved, checking)
        assert(np.all(np.logical_or(np.equal(new_board,board), np.equal(new_board, solved))) == checking)

In [5]:
def generate_sudokus(board, solved, moves_per_board, correct_moves_per_board):
    boards = []
    labels = []
    
    n_moves = len(np.argwhere(board == 0))
    for move in n_moves:
        
        
        
    return boards,labels

generate_sudokus(board, solved, 1, 1)

IndentationError: expected an indented block (<ipython-input-5-b9b55a1f5677>, line 10)

In [ ]:
import pickle

In [ ]:
with open('data/sudoku_lists/%d_sudokus.pil' % 101, 'rb') as handle:
    sudokus = pickle.load(handle)

In [7]:
board

array([[0, 0, 0, 0, 0, 0, 0, 1, 0],
       [4, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 5, 0, 4, 0, 7],
       [0, 0, 8, 0, 0, 0, 3, 0, 0],
       [0, 0, 1, 0, 9, 0, 0, 0, 0],
       [3, 0, 0, 4, 0, 0, 2, 0, 0],
       [0, 5, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 8, 0, 6, 0, 0, 0]], dtype=object)

In [8]:
new_board = sudoku_utils.make_random_move(board, solved)

In [13]:
np.sum(~np.equal(board, new_board))

1